### AI 기반 경구 약제 이미지 식별 모델

<br>

[경구약제 이미지 데이터](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=576)
<br>[AI 기반 경구 약제 이미지 식별 모델](https://aifactory.space/task/2636/overview)

Deveopment Envrionment
<br>TensorflowHub Model
<br>Keras Applications Model
<br>CNN Model

<br>

### Development Environment

In [ ]:
!pip install tensorflow_hub
!pip install tensorflow==2.10.0

In [1]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Teansorflow:", tf.__version__)
print()
print(tf.test.is_built_with_cuda())
print()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print()
print(tf.test.gpu_device_name())
print()
print(device_lib.list_local_devices())

2023-11-12 23:54:27.036912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 23:54:27.036942: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 23:54:27.036972: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 23:54:27.044225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 23:54:27.774708: W tensorflow/c

Teansorflow: 2.14.0

True

Num GPUs Available:  2

/device:GPU:0

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10626029690929436185
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23483842560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13043100682625658280
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23483842560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15342944436273729242
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6"
xla_global_id: 2144165316
]


2023-11-12 23:54:29.528920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:0 with 22395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-11-12 23:54:29.529565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:1 with 22395 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6
2023-11-12 23:54:29.532031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:0 with 22395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-11-12 23:54:29.532159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /device:GPU:1 with 22395 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

### Tensorflow Hub Model

In [3]:
# https://www.tensorflow.org/hub/tutorials/image_classification?hl=ko
image_size = 128
dynamic_size = False

model_name = "imagenet/mobilenet_v1_100_128/classification" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/classification/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/classification/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/classification/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/classification/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/classification/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/classification/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/classification/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/classification/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/classification/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/classification/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/classification/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/classification/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/classification/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/classification/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/classification/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/classification/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/classification/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/classification/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/classification/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/classification/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/classification/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/classification/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/classification/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/classification/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/classification/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/classification/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/ilsvrc2012_classification/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/classification/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/classification/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/classification/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/classification/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/classification/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/classification/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/classification/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/classification/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/classification/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/classification/5",
  "imagenet/mobilenet_v1_025_128/classification": "https://tfhub.dev/google/imagenet/mobilenet_v1_025_128/classification/5",
  "imagenet/mobilenet_v2_050_128/classification": "https://tfhub.dev/google/imagenet/mobilenet_v2_050_128/classification/5",
  "imagenet/mobilenet_v1_100_128/classification": "https://tfhub.dev/google/imagenet/mobilenet_v1_100_128/classification/5"
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "mobilenet_v2_100_224": 224,
  "mobilenet_v2_130_224": 224,
  "mobilenet_v2_140_224": 224,
  "nasnet_large": 331,
  "nasnet_mobile": 224,
  "pnasnet_large": 331,
  "resnet_v1_50": 224,
  "resnet_v1_101": 224,
  "resnet_v1_152": 224,
  "resnet_v2_50": 224,
  "resnet_v2_101": 224,
  "resnet_v2_152": 224,
  "mobilenet_v3_small_100_224": 224,
  "mobilenet_v3_small_075_224": 224,
  "mobilenet_v3_large_100_224": 224,
  "mobilenet_v3_large_075_224": 224,
  "imagenet/mobilenet_v1_025_128/classification": 128,
  "imagenet/mobilenet_v2_050_128/classification": 128,
  "imagenet/mobilenet_v1_100_128/classification": 128
}

model_handle = model_handle_map[model_name]

print(f"Selected model: {model_name} : {model_handle}")
classifier = hub.load(model_handle)

Selected model: imagenet/mobilenet_v1_100_128/classification : https://tfhub.dev/google/imagenet/mobilenet_v1_100_128/classification/5


2023-11-09 07:22:28.689264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22451 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-11-09 07:22:28.689422: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22451 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [4]:
IMAGE_SHAPE = (128, 128)
do_fine_tuning = True
# len(class_names) = 2
print("Building model with", model_handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SHAPE + (3,)),
    hub.KerasLayer(model_handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(2,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SHAPE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v1_100_128/classification/5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              4254889   
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 2)                 2004      
                                                                 
Total params: 4,256,893
Trainable params: 4,235,005
Non-trainable params: 21,888
_________________________________________________________________


In [5]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

train_x_path = 'data/x_train.npy'
train_y_path = 'data/y_train.npy'
test_x_path = 'data/x_test.npy'

x_train = np.load(train_x_path)
y_train = np.load(train_y_path)
y_train = to_categorical(y_train)
x_test = np.load(test_x_path)

checkpoint_path = "model/cp-{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq='epoch', 
                                                 verbose=1)

In [9]:
model.fit(x_train, y_train, epochs=5, batch_size=16, callbacks=[cp_callback])

Epoch 1/5
  1/700 [..............................] - ETA: 27:11 - loss: 1.4897 - accuracy: 0.7500

2023-11-09 01:16:46.758344: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8900
2023-11-09 01:16:46.859677: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


699/700 [============================>.] - ETA: 0s - loss: 1.2517 - accuracy: 0.8790
Epoch 1: saving model to model/cp-01.ckpt
700/700 [==============================] - 15s 19ms/step - loss: 1.2507 - accuracy: 0.8790
Epoch 2/5
700/700 [==============================] - ETA: 0s - loss: 0.5079 - accuracy: 0.9601
Epoch 2: saving model to model/cp-02.ckpt
700/700 [==============================] - 13s 18ms/step - loss: 0.5079 - accuracy: 0.9601
Epoch 3/5
699/700 [============================>.] - ETA: 0s - loss: 0.4805 - accuracy: 0.9736
Epoch 3: saving model to model/cp-03.ckpt
700/700 [==============================] - 13s 19ms/step - loss: 0.4804 - accuracy: 0.9737
Epoch 4/5
700/700 [==============================] - ETA: 0s - loss: 0.4640 - accuracy: 0.9818
Epoch 4: saving model to model/cp-04.ckpt
700/700 [==============================] - 13s 18ms/step - loss: 0.4640 - accuracy: 0.9818
Epoch 5/5
700/700 [==============================] - ETA: 0s - loss: 0.4572 - accuracy: 0.9858
Epo

In [5]:
epoch = 5
for i in range(1, epoch + 1):
    IMAGE_SHAPE = (128, 128)
    do_fine_tuning = True
    # len(class_names) = 2
    print("Building model with", model_handle)
    load_model = tf.keras.Sequential([
        # Explicitly define the input shape so the model can be properly
        # loaded by the TFLiteConverter
        tf.keras.layers.InputLayer(input_shape=IMAGE_SHAPE + (3,)),
        hub.KerasLayer(model_handle, trainable=do_fine_tuning),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(2,
                            kernel_regularizer=tf.keras.regularizers.l2(0.0001))
    ])
    load_model.build((None,)+IMAGE_SHAPE+(3,))
    # load_model.summary()
    result_csv_path = 'log/predicted_classes_epoch_' + str(i) + '.csv'
    if i < 10:

        load_model.load_weights('model/cp-0'+ str(i) + '.ckpt')
    elif i >= 10 and i < 100:
        load_model.load_weights('model/cp-'+ str(i) + '.ckpt')       
    predictions = load_model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    predicted_classes_df = pd.DataFrame(predicted_classes, index=None)
    predicted_classes_df.to_csv(result_csv_path, index=False)

Building model with https://tfhub.dev/google/imagenet/mobilenet_v1_100_128/classification/5
 11/150 [=>............................] - ETA: 0s 

2023-11-09 07:22:41.114967: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8900
2023-11-09 07:22:41.196782: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


150/150 [==============================] - 1s 6ms/step


### Keras Applications Model

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [7]:
img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1)
    ],
    
    name = 'img_augmentation'
)

In [ ]:
train_x_path = 'data/x_train.npy'
train_y_path = 'data/y_train.npy'
test_x_path = 'data/x_test.npy'

x_train = np.load(train_x_path)
y_train = np.load(train_y_path)
y_train = to_categorical(y_train)
x_test = np.load(test_x_path)

img_size = (224, 224)
NUM_CLASSES = 2

inputs = tf.keras.layers.Input(shape=(img_size[0], img_size[1], 3))
x = img_augmentation(inputs)
outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(x)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

checkpoint_path = "model/cp-{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
epoch = 10

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq='epoch', 
                                                 verbose=1)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epoch, batch_size=32, callbacks=[cp_callback])

In [ ]:
for i in range(1, epoch + 1):

    img_size = (224, 224)
    NUM_CLASSES = 2

    inputs = tf.keras.layers.Input(shape=(img_size[0], img_size[1], 3))
    x = img_augmentation(inputs)
    outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(x)

    load_model = tf.keras.Model(inputs, outputs)
    load_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )

    load_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # load_model.summary()
    result_csv_path = 'log/predicted_classes_epoch_' + str(i) + '.csv'
    if i < 10:
        load_model.load_weights('model/cp-0'+ str(i) + '.ckpt')
    elif i >= 10 and i < 100:
        load_model.load_weights('model/cp-'+ str(i) + '.ckpt')       
    predictions = load_model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    predicted_classes_df = pd.DataFrame(predicted_classes, index=None)
    predicted_classes_df.to_csv(result_csv_path, index=False)

### CNN Model

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

train_x_path = 'data/x_train.npy'
train_y_path = 'data/y_train.npy'
test_x_path = 'data/x_test.npy'

x_train = np.load(train_x_path)
y_train = np.load(train_y_path)
y_train = to_categorical(y_train)
x_test = np.load(test_x_path)

img_size = (128, 128)

model = tf.keras.models.Sequential([
    layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer

    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer

    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(y_train.shape[1], activation = 'sigmoid') # ouput layer
    ])

checkpoint_path = "model/cp-{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
epoch = 10

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq='epoch', 
                                                 verbose=1)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epoch, batch_size=32, callbacks=[cp_callback])

2023-11-12 23:54:37.879187: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22395 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-11-12 23:54:37.879366: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22395 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


Epoch 1/10


2023-11-12 23:54:41.008285: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-12 23:54:41.341765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-11-12 23:54:43.620509: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7efbc5e32f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-12 23:54:43.620536: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-12 23:54:43.620543: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-12 23:54:43.625526: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR cra

350/350 [==============================] - ETA: 0s - loss: 0.5009 - accuracy: 0.7887
Epoch 1: saving model to model/cp-01.ckpt
350/350 [==============================] - 32s 67ms/step - loss: 0.5009 - accuracy: 0.7887
Epoch 2/10
350/350 [==============================] - ETA: 0s - loss: 0.2790 - accuracy: 0.8892
Epoch 2: saving model to model/cp-02.ckpt
350/350 [==============================] - 24s 67ms/step - loss: 0.2790 - accuracy: 0.8892
Epoch 3/10
350/350 [==============================] - ETA: 0s - loss: 0.1876 - accuracy: 0.9295
Epoch 3: saving model to model/cp-03.ckpt
350/350 [==============================] - 24s 68ms/step - loss: 0.1876 - accuracy: 0.9295
Epoch 4/10
350/350 [==============================] - ETA: 0s - loss: 0.1522 - accuracy: 0.9437
Epoch 4: saving model to model/cp-04.ckpt
350/350 [==============================] - 24s 67ms/step - loss: 0.1522 - accuracy: 0.9437
Epoch 5/10
350/350 [==============================] - ETA: 0s - loss: 0.1255 - accuracy: 0.9550

In [4]:
for i in range(1, epoch + 1):
    IMAGE_SHAPE = (128, 128)

    load_model = tf.keras.models.Sequential([
    layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer

    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    tf.keras.layers.BatchNormalization(), #batch norm layer

    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(y_train.shape[1], activation = 'sigmoid') # ouput layer
    ])

    load_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # load_model.summary()
    result_csv_path = 'log/predicted_classes_epoch_' + str(i) + '.csv'
    if i < 10:
        load_model.load_weights('model/cp-0'+ str(i) + '.ckpt')
    elif i >= 10 and i < 100:
        load_model.load_weights('model/cp-'+ str(i) + '.ckpt')       
    predictions = load_model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    predicted_classes_df = pd.DataFrame(predicted_classes, index=None)
    predicted_classes_df.to_csv(result_csv_path, index=False)

150/150 [==============================] - 3s 16ms/step


### Reference

<b>Tensorflow</b>
<br>[Retraining an Image Classifier](https://www.tensorflow.org/hub/tutorials/tf2_image_retraining)

<b>Dacon</b>
<br>[전처리 및 CNN 코드 설계하기 (0.8808)](https://dacon.io/competitions/official/235874/codeshare/4567)